This file contains
1.simple model using ollama
2.RAG model using ollama
3.simple model using groq
4.RAG model using Groq

Setup wise, you likley to run into some errors. 
For ollama go to ollama.com downlaod the app and pull the models you want to use. This will take significant space so be weary.
For groq youll simply loop up groq api and make an account then put your api key in the api_key space, this runs faster but I did run out of alloated calls when doing some of my hw before. 


In [2]:
import os
!pip install langchain sentence-transformers transformers ollama
from langchain.llms import Ollama
import warnings
import time
import requests
import warnings
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings
from PyPDF2 import PdfReader
from bs4 import BeautifulSoup

from groq import Groq


In [7]:
ollama_model = Ollama(model="taozhiyuai/openbiollm-llama-3:70b_q2_k", temperature=0) 

def ask_question(question):
    response = ollama_model.invoke(question)
    return response

question = "What does Nutritional deficiencies and excesses arise from "
answer = ask_question(question)
print("Question:", question)
print("Answer:", answer)

OllamaEndpointNotFoundError: Ollama call failed with status code 404. Maybe your model is not found and you should pull the model with `ollama pull taozhiyuai/openbiollm-llama-3:70b_q2_k`.

In [4]:
def load_pdf_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        with open("temp.pdf", "wb") as f:
            f.write(response.content)
        reader = PdfReader("temp.pdf")
        text = "".join([page.extract_text() for page in reader.pages])
        return text
    except requests.RequestException as e:
        print(f"Failed to download PDF from {url}: {str(e)}")
        return ""

def load_data():
    pdf_urls = [
        "https://www.biorxiv.org/content/10.1101/2020.07.28.224253v1.full.pdf",
        "https://www.cartercenter.org/resources/pdfs/health/ephti/library/lecture_notes/health_extension_trainees/generalpathology.pdf"
    ]

    documents_with_metadata = []
    for url in pdf_urls:
        data = load_pdf_from_url(url)
        if data:
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
            splits = text_splitter.split_text(data)

            for split in splits:
                documents_with_metadata.append(Document(page_content=split, metadata={"source": url}))

    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    if not os.path.exists("chromadb"):
        os.makedirs("chromadb")

    vectorstore = Chroma.from_documents(
        documents=documents_with_metadata, 
        embedding=embedding_model, 
        persist_directory="chromadb"
    )

    return vectorstore

vectorstore = load_data()

ollama_model = Ollama(model="llama3", temperature=0)

KeyboardInterrupt: 

In [ ]:
def get_answer(query):
    template = """
    Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know; don't try to make up an answer.
    Keep the answer as concise as possible, preferably in three sentences.
    {context}
    Question: {question}
    Answer:
    """

    QA_CHAIN_PROMPT = PromptTemplate(
        input_variables=["context", "question"],
        template=template,
    )

    retriever = vectorstore.as_retriever()
    context = retriever.invoke(query)
    context_text = "\n".join([doc.page_content for doc in context])

    prompt = QA_CHAIN_PROMPT.format(context=context_text, question=query)

    response = ollama_model.invoke(prompt)
    return response


question = "What is Laceration versus incision"
answer = get_answer(question)
print("Question:", question)
print("Answer:", answer)

In [ ]:
api_key = "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"
#your api key


groq_client = Groq(api_key=api_key)

def query_llama3(question):
    response = groq_client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": question
            }
        ],
        model="llama3-8b-8192"
    )
    return response.choices[0].message.content

question = "What does Nutritional deficiencies and excesses arise from"
answer = query_llama3(question)
print("Question:", question)
print("Answer:", answer)

In [ ]:
api_key = "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"
#your api key

groq_client = Groq(api_key=api_key)

def load_pdf_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        with open("temp.pdf", "wb") as f:
            f.write(response.content)
        reader = PdfReader("temp.pdf")
        text = "".join([page.extract_text() for page in reader.pages])
        return text
    except requests.RequestException as e:
        print(f"Failed to download PDF from {url}: {str(e)}")
        return ""

def load_data():
    pdf_urls = [
        "https://www.biorxiv.org/content/10.1101/2020.07.28.224253v1.full.pdf",
        "https://www.cartercenter.org/resources/pdfs/health/ephti/library/lecture_notes/health_extension_trainees/generalpathology.pdf"
    ]
   
    pdf_urls.extend(scrape_biorxiv_pdf_urls())
    
    documents_with_metadata = []
    for url in pdf_urls:
        data = load_pdf_from_url(url)
        if data:
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
            splits = text_splitter.split_text(data)

            for split in splits:
                documents_with_metadata.append(Document(page_content=split, metadata={"source": url}))

    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    if not os.path.exists("chromadb"):
        os.makedirs("chromadb")

    vectorstore = Chroma.from_documents(
        documents=documents_with_metadata, 
        embedding=embedding_model, 
        persist_directory="chromadb"
    )

    return vectorstore

vectorstore = load_data()

In [ ]:
#New model with both PDF and website 
def scrape_biorxiv_pdf_urls():
    base_url = "https://www.biorxiv.org"
    pdf_urls = []

    # Send a GET request to the biorxiv main page or a specific section
    response = requests.get(base_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")

        # Find all PDF download links with the specified class
        pdf_links = soup.find_all("a", class_="article-dl-pdf-link")
        
        for link in pdf_links:
            href = link.get("href")
            if href and href.endswith(".pdf"):
                # Construct the full URL and add it to the list
                full_url = base_url + href
                pdf_urls.append(full_url)
    
    return pdf_urls


In [ ]:
api_key = "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"
groq_client = Groq(api_key=api_key)

def load_pdf_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        with open("temp.pdf", "wb") as f:
            f.write(response.content)
        reader = PdfReader("temp.pdf")
        text = "".join([page.extract_text() for page in reader.pages])
        return text
    except requests.RequestException as e:
        print(f"Failed to download PDF from {url}: {str(e)}")
        return ""

def scrape_website(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        paragraphs = soup.find_all('p')
        content = "\n".join([para.get_text() for para in paragraphs])
        return content
    except requests.RequestException as e:
        print(f"Failed to scrape website {url}: {str(e)}")
        return ""

def load_data():

    pdf_urls = [
        "https://www.biorxiv.org/content/10.1101/2020.07.28.224253v1.full.pdf",
        "https://www.cartercenter.org/resources/pdfs/health/ephti/library/lecture_notes/health_extension_trainees/generalpathology.pdf"
    ]

    
    website_urls = [
        "https://en.wikipedia.org/wiki/Pathology",
        "https://www.mcgill.ca/pathology/about/definition#:~:text=Pathology%20is%20a%20branch%20of,the%20whole%20body%20(autopsy)."
    ]

    documents_with_metadata = []

    for url in pdf_urls:
        data = load_pdf_from_url(url)
        if data:
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
            splits = text_splitter.split_text(data)
            for split in splits:
                documents_with_metadata.append(Document(page_content=split, metadata={"source": url}))


    for url in website_urls:
        data = scrape_website(url)
        if data:
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
            splits = text_splitter.split_text(data)
            for split in splits:
                documents_with_metadata.append(Document(page_content=split, metadata={"source": url}))

    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


    if not os.path.exists("chromadb"):
        os.makedirs("chromadb")

    vectorstore = Chroma.from_documents(
        documents=documents_with_metadata, 
        embedding=embedding_model, 
        persist_directory="chromadb"
    )

    return vectorstore

# Load the data
vectorstore = load_data()

In [ ]:
def query_rag(question):

    template = """
    Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know; don't try to make up an answer.
    Keep the answer as concise as possible, preferably in three sentences.
    Quote the url the info came from.
    {context}
    Question: {question}
    Answer:
    """


    retriever = vectorstore.as_retriever()
    context_docs = retriever.get_relevant_documents(question)
    context_text = "\n".join([doc.page_content for doc in context_docs])


    prompt = template.format(context=context_text, question=question)


    response = groq_client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        model="llama3-8b-8192"
    )


    return response.choices[0].message.content


question = "Finish this sentance, The study of pathology, including the detailed examination of the body,"
answer = query_rag(question)
print("Question:", question)
print("Answer:", answer)
